In [1]:
import tensorflow as tf
import numpy as np

#set hyperparameters
max_len = 40
step = 2
num_units = 128
learning_rate = 0.001
batch_size = 200
epoch = 2
temperature = 0.5

In [2]:
def read_data(file_name):
    '''
     open and read text file
    '''
    text = open(file_name, 'r').read()
    return text.lower()

In [3]:
def featurize(text):
    '''
     featurize the text to train and target dataset
    '''
    unique_chars = list(set(text))
    len_unique_chars = len(unique_chars)

    input_chars = []
    output_char = []

    for i in range(0, len(text) - max_len, step):
        input_chars.append(text[i:i+max_len])
        output_char.append(text[i+max_len])

    train_data = np.zeros((len(input_chars), max_len, len_unique_chars))
    target_data = np.zeros((len(input_chars), len_unique_chars))

    for i , each in enumerate(input_chars):
        for j, char in enumerate(each):
            train_data[i, j, unique_chars.index(char)] = 1
        target_data[i, unique_chars.index(output_char[i])] = 1
    return train_data, target_data, unique_chars, len_unique_chars


In [4]:
def rnn(x, weight, bias, len_unique_chars):
    '''
     define rnn cell and prediction
    '''
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, len_unique_chars])
    x = tf.split(x, max_len, 0)

    cell = tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction

In [5]:
def sample(predicted):
    '''
     helper function to sample an index from a probability array
    '''
    exp_predicted = np.exp(predicted/temperature)
    predicted = exp_predicted / np.sum(exp_predicted)
    probabilities = np.random.multinomial(1, predicted, 1)
    return probabilities


In [6]:
def run(train_data, target_data, unique_chars, len_unique_chars):
    '''
     main run function
    '''
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)

    num_batches = int(len(train_data)/batch_size)

    for i in range(epoch):
        print "----------- Epoch {0}/{1} -----------".format(i+1, epoch)
        count = 0
        for _ in range(num_batches):
            train_batch, target_batch = train_data[count:count+batch_size], target_data[count:count+batch_size]
            count += batch_size
            sess.run([optimizer] ,feed_dict={x:train_batch, y:target_batch})
        #get on of training set as seed
        seed = train_batch[:1:]
        save_path = saver.save(sess, "./tmp/rnn_model" + str(i) + ".ckpt")

        #to print the seed 40 characters
        seed_chars = ''
        for each in seed[0]:
                seed_chars += unique_chars[np.where(each == max(each))[0][0]]
        print "Seed:", seed_chars

        #predict next 1000 characters
        for i in range(1000):
            if i > 0:
                remove_fist_char = seed[:,1:,:]
                seed = np.append(remove_fist_char, np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
            predicted = sess.run([prediction], feed_dict = {x:seed})
            predicted = np.asarray(predicted[0]).astype('float64')[0]
            probabilities = sample(predicted)
            predicted_chars = unique_chars[np.argmax(probabilities)]
            seed_chars += predicted_chars
        print 'Result:', seed_chars
    sess.close()


In [7]:
def convert_to_seed(text, unique_chars):
    len_uniques = len(unique_chars)
    letters = []
    for i in text:
        letter = np.zeros((1, len_uniques))
        for j in range(len(unique_chars)):
            if i == unique_chars[j]:
                letter[0, j] = 1.
        letters.append(letter)
    return np.array([np.vstack(letters)])
    

In [8]:
def restore_model():
    text = read_data('shakespeare.txt')
    train_data, target_data, unique_chars, len_unique_chars = featurize(text)
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver()    
    sess = tf.Session()
    sess.run(init_op)
    saver.restore(sess, "./tmp/rnn_model1.ckpt")
    
    return unique_chars, prediction, sess, x, len_unique_chars

In [9]:
def predict(text, unique_chars, prediction, sess, x, len_unique_chars):
    
    text_head = ''
    pad_head = 0
    if len(text) > 40:
        text_head = text[:-40]        
        text = text[-40:]
    elif len(text) < 40:
        pad_head  = 40 - len(text)
        text = ' ' * pad_head + text
        
    seed = convert_to_seed(text, unique_chars)
    #to print the seed 40 characters
    seed_chars = ''
    for each in seed[0]:
            seed_chars += unique_chars[np.where(each == max(each))[0][0]]
    print "Seed:", seed_chars

    
    #predict next 1000 characters
    for i in range(50):
        if i > 0:
            remove_fist_char = seed[:,1:,:]
            seed = np.append(remove_fist_char, 
                             np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
        predicted = sess.run([prediction], feed_dict = {x:seed})
        predicted = np.asarray(predicted[0]).astype('float64')[0]
        probabilities = sample(predicted)
        predicted_chars = unique_chars[np.argmax(probabilities)]
        seed_chars += predicted_chars
        if predicted_chars == '.':
            return (text_head + seed_chars)[pad_head:]

    

In [10]:
# unique_chars, predictions, sess, x, len_unique_chars = restore_model()
# predict('out fane with you ', unique_chars, predictions, sess, x, len_unique_chars)

In [11]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# run(train_data, target_data, unique_chars, len_unique_chars)

In [12]:
# text = read_data('shakespeare.txt')
# train_data, target_data, unique_chars, len_unique_chars = featurize(text)
# train_batch, target_batch = train_data[1:5], target_data[1:5]
# seed = train_batch[:1:]
# print seed

In [13]:
# np.zeros((1, 5))

In [14]:
# def convert_to_seed(text, unique_chars):
#     len_uniques = len(unique_chars)
#     letters = []
#     for i in text:
#         letter = np.zeros((1, len_uniques))
#         for j in range(len(unique_chars)):
#             if i == unique_chars[j]:
#                 letter[0, j] = 1.
#         letters.append(letter)
#     return np.array([np.vstack(letters)])
    

In [15]:
# convert_to_seed('hello', unique_chars).shape

In [16]:
text = read_data('shakespeare.txt')
train_data, target_data, unique_chars, len_unique_chars = featurize(text)
run(train_data, target_data, unique_chars, len_unique_chars)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

----------- Epoch 1/2 -----------
Seed: ve him some relief if it be but for that
Result: ve him some relief if it be but for that the sang i sar ave , is a a mant .
the sirn of as bing and .
hare , sar'st i be with the ore master .
thou dast the forther .
if at a and mear of me .
the wird , that ray faries bear the carm .
and his a drack the his manger .
and it thes a mine the eart hat not asting thes inge .
the cront the king and the i so fot not .
i have dow not laing the coming .
ow come , and not the with .
what lough have it the carime .
lot it will came to with the 'tring the bearing , the could mane seer .
ard the is make the i mase .
whore sur him some that dout here and wall my lave and and matir the wast and the faring the ricomed in us not me thou hame dear .
the 'ttre i but best is a , an